In [1]:
import xarray as xr
import pandas as pd
import os
import numpy as np
import pyarrow.parquet as pq
import polars as pl

In [2]:
test = xr.open_dataset("C:/Users/flapet/OneDrive - NOC/Documents/IDAPro/lib/db_building/data/satellite/par_mapped/AQUA_MODIS.20240101.L3m.DAY.PAR.x_par.nc")

Open the PAR ncdfs into one xarray data object, with date as index

In [2]:
# Directory where your NetCDF files are stored (daily files)
ncdf_dir = "C:/Users/flapet/OneDrive - NOC/Documents/IDAPro/lib/db_building/data/satellite/par_tot/"

# List all NetCDF files for the year (one file per day)
ncdf_files = sorted([f for f in os.listdir(ncdf_dir) if f.endswith('.nc')])

# Load the PAR data for each file and store it
daily_par_data = {}
for file in ncdf_files:
    # Open the dataset for the specific day
    ds = xr.open_dataset(os.path.join(ncdf_dir, file))
    
    # Extract PAR data (assuming it's named 'par', adjust if necessary)
    # Add a date key to use as the dictionary key for each day
    date = file.split('.')[1] # Extract the date from the filename (adjust based on filename format)
    
    # Store the data in a dictionary, with the date as the key
    daily_par_data[date] = ds["par"]  # Replace 'par' with the actual variable name in your files

# Example: inspect one of the datasets
print(daily_par_data["20240101"])  # Inspect the PAR data for January 1st, 2024


<xarray.DataArray 'par' (lat: 573, lon: 1063)> Size: 2MB
[609099 values with dtype=float32]
Coordinates:
  * lat      (lat) float32 2kB 71.0 70.96 70.92 70.87 ... 47.29 47.24 47.2 47.16
  * lon      (lon) float32 4kB -43.57 -43.53 -43.49 ... 0.5958 0.6375 0.6792
Attributes:
    long_name:      Photosynthetically Available Radiation, R. Frouin
    units:          mol m^-2 day^-1
    standard_name:  surface_downwelling_photosynthetic_photon_flux_in_air
    valid_min:      -32750
    valid_max:      32250
    reference:      Frouin, R., Ligner, D.W., and Gautier, C., 1989: A Simple...
    display_scale:  linear
    display_min:    0.0
    display_max:    76.2


Open BGC Argo parquet and summarise to one line per profile

In [6]:
#argo = pq.read_table('C:/Users/flapet/OneDrive - NOC/Documents/IDAPro/lib/db_building/data/argo_pq/biocarbon_floats_table.parquet')
argo = pq.read_table('C:/Users/flapet/OneDrive - NOC/Documents/IDAPro/lib/db_building/data/argo_pq/float_icb_fluorescence_cleaned.parquet')
obs_data = argo.to_pandas()

In [7]:
obs_data.head()

,prof_id,depth,LONGITUDE,LATITUDE,PRES,TEMP,PSAL,PSAL_QC,CHLA_ADJUSTED,CHLA_ADJUSTED_QC,BBP700_ADJUSTED,BBP700_ADJUSTED_QC,DOWNWELLING_PAR,DOWN_IRRADIANCE490,NITRATE_ADJUSTED,NITRATE_ADJUSTED_QC,DOXY_ADJUSTED,DOXY_ADJUSTED_QC,JULD_date,JULD
0,2013-04-17 21:49:00-20.617265_,0.0,-20.617265,62.724418,NaN,7.983,35.151001,NaN,0.31025,NaN,0.000581,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,2013-04-17 20:49:00
1,2013-04-17 21:49:00-20.617265_,1.0,-20.617265,62.724418,NaN,7.983,35.151001,NaN,0.31025,NaN,0.000581,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,2013-04-17 20:49:00
2,2013-04-17 21:49:00-20.617265_,2.0,-20.617265,62.724418,NaN,7.983,35.151001,NaN,0.31025,NaN,0.000581,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,2013-04-17 20:49:00
3,2013-04-17 21:49:00-20.617265_,3.0,-20.617265,62.724418,NaN,7.983,35.151001,NaN,0.31025,NaN,0.000581,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,2013-04-17 20:49:00
4,2013-04-17 21:49:00-20.617265_,4.0,-20.617265,62.724418,NaN,7.983,35.151001,NaN,0.31025,NaN,0.000581,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,2013-04-17 20:49:00


In [8]:
obs_data = obs_data[obs_data['JULD'] < pd.to_datetime("2025-01-01")]

In [9]:
def mad_despike(series, threshold=3.5):
    """ Remove spikes using MAD method """
    median = np.nanmedian(series)
    mad = np.nanmedian(np.abs(series - median))
    modified_z_score = 0.6745 * (series - median) / mad if mad else np.zeros_like(series)
    return series[np.abs(modified_z_score) < threshold]


In [10]:
def despike_group(group):
    group["bbp700_cleaned"] = mad_despike(group["BBP700_ADJUSTED"])
    return group



In [11]:
obs_data = obs_data.groupby(["prof_id"], group_keys=False).apply(despike_group)

C:\Users\flapet\AppData\Local\Temp\ipykernel_24300\3163570509.py:3: RuntimeWarning: All-NaN slice encountered
  median = np.nanmedian(series)
C:\Users\flapet\AppData\Local\Temp\ipykernel_24300\3163570509.py:4: RuntimeWarning: All-NaN slice encountered
  mad = np.nanmedian(np.abs(series - median))
C:\Users\flapet\AppData\Local\Temp\ipykernel_24300\3163570509.py:3: RuntimeWarning: All-NaN slice encountered
  median = np.nanmedian(series)
C:\Users\flapet\AppData\Local\Temp\ipykernel_24300\3163570509.py:4: RuntimeWarning: All-NaN slice encountered
  mad = np.nanmedian(np.abs(series - median))
C:\Users\flapet\AppData\Local\Temp\ipykernel_24300\3163570509.py:3: RuntimeWarning: All-NaN slice encountered
  median = np.nanmedian(series)
C:\Users\flapet\AppData\Local\Temp\ipykernel_24300\3163570509.py:4: RuntimeWarning: All-NaN slice encountered
  mad = np.nanmedian(np.abs(series - median))
C:\Users\flapet\AppData\Local\Temp\ipykernel_24300\3163570509.py:3: RuntimeWarning: All-NaN slice encounte

In [14]:
from scipy.spatial import cKDTree

# Function to find the nearest PAR value for each observation
def get_nearest_par(lat, lon, date, daily_par_data):
    # Get the PAR data for the specific date
    if date in daily_par_data and daily_par_data[date].size > 0:
        par_data = daily_par_data[date]
    # Extract the PAR value at the nearest point
        nearest_par = par_data.sel(lon = lon, lat = lat, method = 'nearest').values.flatten()[0]
    else:
        nearest_par = 0
    return nearest_par

# Apply the function to each row in the observation data
obs_data["satellite_par"] = obs_data.apply(
    lambda row: get_nearest_par(row["LATITUDE"], row["LONGITUDE"], row["JULD"].strftime('%Y%m%d'), daily_par_data),
    axis=1
)


Regularize data every meter

In [15]:
# Step 1: Round the `PRES` column to the nearest integer
obs_data['PRES_rounded'] = obs_data['depth']

# Step 2: Group by the rounded `PRES` values
# Aggregate other columns by taking the mean for each group
df_regularized = obs_data

# Step 3: Drop the original `PRES` column if not needed
df_regularized = df_regularized.drop(columns=['PRES'])

df_regularized['year'] = df_regularized['JULD'].dt.year
df_regularized['month'] = df_regularized['JULD'].dt.month
df_regularized['day'] = df_regularized['JULD'].dt.day

Apply the primary production algorythm

In [16]:
df_regularized['BBP470'] = df_regularized['BBP700_ADJUSTED']/(470/400) 
df_regularized['carbon'] = 12128 * df_regularized['BBP470'] + 0.59

In [17]:
def running_mean(array, window_size=5):
    kernel = np.ones(window_size) / window_size
    smoothed_array = np.convolve(array, kernel, mode='same')  # 'same' ensures the output matches input size
    return smoothed_array

In [19]:
from cbpm_argo import cbpm_argo
import numpy as np
from scipy.interpolate import interp1d

dfs = []
depth_grid = np.arange(0,200)

# Iterate through each unique 'JULD' (day)
for i in df_regularized['JULD'].unique():
    # Filter for rows corresponding to the current 'JULD'
    temp_df = df_regularized[df_regularized['JULD'] == i].iloc[0:200,].copy()  # Use `.copy()` to avoid warnings

    # Extract the pressure and chlorophyll values for interpolation
    pres_values = temp_df['PRES_rounded'].to_numpy()
    chl_values = temp_df['CHLA_ADJUSTED'].to_numpy()
    carbon_values = temp_df['carbon'].to_numpy()

    # Apply the running mean smoothing
    chl_smoothed = running_mean(chl_values, window_size=5)
    carbon_smoothed = running_mean(carbon_values, window_size=5)

    # Check for valid data before interpolation (avoid NaN values)
    mask = ~np.isnan(pres_values) & ~np.isnan(chl_values)
    pres_values = pres_values[mask]
    chl_values = chl_values[mask]
    carbon_values = carbon_values[mask]

    # Interpolate the CHLA_ADJUSTED onto the depth grid (0 to 199)
    if len(pres_values) > 1:  # Ensure there's enough data to interpolate
        interpolator = interp1d(pres_values, chl_values, bounds_error=False, fill_value=np.nan)
        interpolated_chl = interpolator(depth_grid)
        
        interpolator = interp1d(pres_values, carbon_values, bounds_error=False, fill_value=np.nan)
        interpolated_carbon = interpolator(depth_grid)
        
    else:
        # If only one point or no valid data, fill with NaN
        print(temp_df['prof_id'].unique())
        interpolated_chl = np.full(depth_grid.shape, np.nan)
        interpolated_carbon = np.full(depth_grid.shape, np.nan)

    # Now we can extract other values and apply the cbpm_argo function
    chl_z = interpolated_chl
    Cphyto_z = interpolated_carbon
    irr = temp_df['satellite_par'].mean()  # Mean irradiance value
    year = int(temp_df['year'].mean())
    month = int(temp_df['month'].mean())
    day = int(temp_df['day'].mean())
    lat = temp_df['LATITUDE'].mean()

    # Call the cbpm_argo function with interpolated data
    [pp_z, mu_z, par_z, prcnt_z, nutTempFunc_z, IgFunc_z, mzeu] = cbpm_argo(chl_z, Cphyto_z, irr, year, month, day, lat)

    size_max = len(temp_df)

    # Use .loc to explicitly assign new columns (expand results back into DataFrame)
    temp_df.loc[:, 'pp'] = pp_z[0:size_max]
    temp_df.loc[:, 'mu'] = mu_z[0:size_max]
    temp_df.loc[:, 'prcnt'] = prcnt_z[0:size_max]
    temp_df.loc[:, 'nutTempFunc'] = nutTempFunc_z[0:size_max]
    temp_df.loc[:, 'IgFunc'] = IgFunc_z[0:size_max]
    temp_df.loc[:, 'zeu'] = np.full(size_max, mzeu)

    # Append modified DataFrame to the list
    dfs.append(temp_df)


# Combine all DataFrames
final_df = pd.concat(dfs)


['2013-05-30 14:32:00-23.3698766666667_']


c:\Users\flapet\OneDrive - NOC\Documents\IDAPro\lib\db_building\script\cbpm_argo.py:139: RuntimeWarning: invalid value encountered in scalar divide
  prcnt_z[z] = par_z[z] / (irr * 0.975)
c:\Users\flapet\OneDrive - NOC\Documents\IDAPro\lib\db_building\script\cbpm_argo.py:81: RuntimeWarning: invalid value encountered in scalar power
  k490 = 0.0166 + 0.0773 * pow(chl_surf,0.6715);


In [20]:
final_df.to_csv('C:/Users/flapet/OneDrive - NOC/Documents/IDAPro/lib/db_building/data/argo_icb_pp_estimations_floats.csv')

In [ ]:
test = df_regularized[df_regularized["PLATFORM_NUMBER"] == "4903532 "]

In [ ]:
test.to_csv('C:/Users/flapet/OneDrive - NOC/Documents/IDAPro/lib/db_building/data/problematic_float.csv')